In [ ]:
print(1+2)

: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:

import yfinance as yf
yf.set_tz_cache_location(None)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ============================
# STEP 1: MARKET DATA
# ============================

tickers = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA", "NVDA", "JPM", "V", "JNJ", "WMT"]

data = yf.download(tickers, start="2023-01-01", end="2024-01-01", progress=False)
data = data["Close"]

returns = data.pct_change().dropna()

mu = returns.mean() * 252
sigma = returns.cov() * 252

mu = mu.to_numpy()
sigma = sigma.to_numpy()

print("\n--- Real Data for QAOA ---")
print("Top 3 Expected Returns:", mu[:3])
print("Covariance Matrix Shape:", sigma.shape)




In [ ]:

# ============================
# STEP 2: BUILD QP MODEL
# ============================
from qiskit_optimization import QuadraticProgram


budget = 4
risk_factor = 0.5
num_assets = len(tickers)

qp = QuadraticProgram()

for i in range(num_assets):
    qp.binary_var(name=tickers[i])

qp.minimize(
    linear = -mu,
    quadratic = risk_factor * sigma
)

qp.linear_constraint(
    linear = {tickers[i]: 1 for i in range(num_assets)},
    sense  = "==",
    rhs    = budget,
    name   = "budget_constraint"
)



In [ ]:

# ============================
# STEP 3: QUBO CONVERSION
# ============================
from qiskit_optimization.converters import QuadraticProgramToQubo

converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)


In [ ]:
# ============================
# STEP 4: QAOA SOLVER (FIXED)
# ============================

from qiskit.algorithms.minimum_eigen_solvers import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import Aer
from qiskit.utils import QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Convert QUBO -> Ising Hamiltonian
op, offset = qubo.to_ising()

# FIX complex coefficients problem
from qiskit.quantum_info import SparsePauliOp
mat = op.to_matrix().real
op = SparsePauliOp.from_operator(mat)

# Track optimization landscape
opt_history = []
def qaoa_callback(eval_count, params, mean, std):
    opt_history.append((eval_count, mean))

backend = Aer.get_backend("aer_simulator")

# QAOA solver
qaoa = QAOA(
    optimizer=COBYLA(maxiter=100),
    reps=1,
    quantum_instance=QuantumInstance(backend),
    callback=qaoa_callback,
)

# Compute eigenvalue with QAOA
eigen_result = qaoa.compute_minimum_eigenvalue(op)

# Interpret result back into a portfolio solution
solver = MinimumEigenOptimizer(qaoa)
result = solver.interpret(eigen_result, qubo)

print("\n--- 4. QAOA RESULTS ---")
selection = result.x
print("Optimal binary solution:", selection)

selected_tickers = [tickers[i] for i, x in enumerate(selection) if x == 1]
print("Selected portfolio:", selected_tickers)

# === Plot optimization landscape ===
steps = [s for s, v in opt_history]
values = [v for s, v in opt_history]

plt.figure()
plt.plot(steps, values, marker="o")
plt.xlabel("Optimizer evaluations")
plt.ylabel("QAOA cost (expected energy)")
plt.title("QAOA optimization landscape")
plt.grid(True)
plt.show()


In [ ]:


# ============================
# STEP 5: RESULTS
# ============================

# Metrics
opt_return = np.dot(mu, selection)
opt_risk   = np.sqrt(selection.T @ sigma @ selection)

print(f"Expected Annual Return: {opt_return:.2%}")
print(f"Annual Volatility (Risk): {opt_risk:.2%}")
print("\n==============================")
print(" Quantum Portfolio Completed ")
